In [ ]:
import pandas as pd
import win32com.client

# 📥: Importing databases.
mb52 = pd.read_excel('MB52.xlsx')
itens_bloqueados = pd.read_excel('Localização de itens.xlsx')
qm10 = pd.read_excel('QM10.xlsx')
contatos = pd.read_excel('Contatos.xlsx')


#🗃️: Treating databases
mb52 = mb52[mb52['Storage Location'] == 'I140']
itens_bloqueados = itens_bloqueados[itens_bloqueados['Almox'] == 'I140']
qm10['Vendor'] = qm10['Vendor'].astype(str).str.lstrip('0')


# 🤝: Joining databases.
merged_df = pd.merge(mb52, itens_bloqueados[['PN', 'QNOTE']], left_on='Material', right_on='PN', how='left')


#📉: Selecting the desired columns.
merged_df['QNOTE'] = merged_df.apply(lambda x: x['QNOTE'] if x['Material'] == x['PN'] else '', axis=1)


# select only the columns of interest
result_df = merged_df[['Material', 'QNOTE']]


#🗑️: Removing rows with empty or null values.
result_df = result_df.dropna(how='any')
result_df = result_df.astype(str)


#create connection with outlook
outlook = win32com.client.Dispatch('outlook.application')


#📤: Sending emails.
for i, material in enumerate(result_df['Material']):
    fornecedor = qm10.loc[qm10['Material'] == material, 'Vendor']
    if fornecedor.empty:
        print("Fornecedor não encontrado para o material:", material)
        continue
    else:
        supplier_matches = contatos['Supplier'].astype(str) == str(fornecedor.values[0])
        if supplier_matches.any():
            nome_fornecedor = contatos.loc[supplier_matches, 'Supplier_Name'].values[0]
            email = str(contatos.loc[supplier_matches, 'Email'].values[0])
            mail = outlook.CreateItem(0)
            mail.To = email
            mail.Subject = 'Alerta de Qualidade Item:{} '.format(material)
            mail.Body = '''
            Time {},
                Estamos pedindo suporte para que seja dada disposição do(s) item(ns) que está(ão) impactando nosso estoque de itens NOK.
                Obs 1: Estamos considerando o prazo de dois dias para resposta deste e-mail, caso não tenhamos resposta vamos devolver os itens e cobrar o frete via nota de débito.
                Obs 2: Sabemos que algumas disposições  já podem ter sido dadas anteriormente, e pedimos desculpas por estar havendo uma nova cobrança, se esse for o caso por favor me envie a disposição em resposta nesse e-mail.  
                Obs 3: Se a disposição for devolução, já me responda com as informações da transportadora que vai coletar.
                ----------------------------------------------------------------------------------------------------------------------
                {} Team,
                We are asking for support to be provided for the item(s) that are impacting our inventory of NOK items.
                Note 1: We are considering two days to respond to this email, if we do not have an answer we will return the items and charge the freight via debit note.
                Note 2: We know that some provisions  they may have already been given, and we apologize for a new charge being made, if that is the case please send me the disposition in response in this email.  
                Note 3: If the disposition is returned, please answer me with the information of the carrier you are going to collect.
            Att.,'''.format(nome_fornecedor,nome_fornecedor)
            attachment = r'C:\Users\f1p1xq8\Downloads\Recuperação de custos.msg'
            mail.Attachments.Add(attachment)
            attachment2 = r'C:\Users\f1p1xq8\Downloads\COPQ_External_Letter.pdf'
            mail.Attachments.Add(attachment2)
            mail.Send()
        else:
            print("Não foi encontrado o fornecedor com o código:", fornecedor)
            continue